In [1]:
# Installing dotenv library to manage confidential keys saved as environment variables
#!pip install python-dotenv

In [2]:
import requests
import json
import pandas as pd
import datetime
import boto3

import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
def timestamp_to_datetime(df, column):
# converts 10-digit timestamp to datetime
    for row in range(0, len(df)):
        df.loc[row, column] = datetime.datetime.fromtimestamp(int(df.loc[row, column]))

In [4]:
def split_dict_to_columns(df, column_to_split):
    # Splits column where each cell is a dictionary 
    # into several columns (one new column per key)
    keys = df[column_to_split][0].keys()
    for row in range(0, len(df)):
        for key in keys:
            df.loc[row, column_to_split+"_"+key] = df[column_to_split][row][key]

In [5]:
# Loading .csv file with city coordinates from S3 bucket
url = 'https://kayak-booking-bucket-12-12-2022.s3.eu-west-3.amazonaws.com/city_coordinates.csv'
city_coord = pd.read_csv(url)
city_coord.head(5)

,place_id,city_name,lat,lon
0,156094680,Mont Saint-Michel,48.635954,-1.511460
1,297756747,Saint-Malo,48.649518,-2.026041
2,297981358,Bayeux,49.276462,-0.702474
3,298137491,Le Havre,49.493898,0.107973
4,297518815,Rouen,49.440459,1.093966


### Getting weather data

In [6]:
results = []

# Loading API key from .env file
API_KEY = os.getenv("API_KEY")

# Using city latitude and longtitude to get weather forecast for next 7 days for each of the cities
for i in range(0, len(city_coord)):
    lat = city_coord.loc[i, "lat"]
    lon = city_coord.loc[i, "lon"]
    response = requests.get(f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&exclude=current,minutely,hourly&units=metric&appid={API_KEY}")
    # We exclude current, minutely and hourly information to focus on daily forecast
    # We indicate that we want to get values expressed in metric units
    results.append(response.json())

In [7]:
# Checking if we obtained information for all 35 cities
print("Lenght of results: ", len(results))
print()

# Checking what information we obtained for a city (commented to avoid long output)
#print("Data obtained for a city: ", results[0])

# Checking the type of data obtained for a city
print("Type of data obtained for a city: ", type(results[0]))
print()

# As information for each one city is a dictionary, let us get a list of keys for future use
print("Dictionary keys: ", results[0].keys())
print()

# Checking information stored as value for the 'daily' key
# (commented to avoid long output)
#print("Value stored in 'daily' key: ", results[0]['daily'])
print()

# The information is stored as a list
print("Data type of value stored in 'daily' key: ", type(results[0]['daily']))
print()

# The length of the list corresponds to the number of days for which the forecast is know
print("Number of days: ",len(results[0]['daily']))
print()

# Weather data for one day in one city
print("Forecast for one day: ", results[0]['daily'][0])
print()

print("Data type for one day of weather data: ", type(results[0]['daily'][0]))

Lenght of results:  35

Type of data obtained for a city:  <class 'dict'>

Dictionary keys:  dict_keys(['lat', 'lon', 'timezone', 'timezone_offset', 'daily'])


Data type of value stored in 'daily' key:  <class 'list'>

Number of days:  8

Forecast for one day:  {'dt': 1677067200, 'sunrise': 1677049333, 'sunset': 1677087438, 'moonrise': 1677053940, 'moonset': 1677098400, 'moon_phase': 0.08, 'temp': {'day': 8.87, 'min': 6.21, 'max': 9.17, 'night': 6.21, 'eve': 8.39, 'morn': 6.85}, 'feels_like': {'day': 6.03, 'night': 3.88, 'eve': 5.9, 'morn': 5.31}, 'pressure': 1014, 'humidity': 93, 'dew_point': 7.63, 'wind_speed': 7.12, 'wind_deg': 330, 'wind_gust': 10.53, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}], 'clouds': 94, 'pop': 0.66, 'rain': 1.25, 'uvi': 1.17}

Data type for one day of weather data:  <class 'dict'>


Weather data for each day is stored in the form of dictionary. We will create a list of such dictionaries and then build a dataframe from this list.

In [8]:

# Creating a list of dictionaries
weather_by_day = []

for city in range(0, len(results)):
    for day in range(0, 7): 
        # keeping weather information about next 7 days
        day_weather = results[city]['daily'][day]
        weather_by_day.append(day_weather)

# Creating a dataframe from a list of dictionaries
weekly_forecast_df = pd.DataFrame.from_records(weather_by_day)

# Checking the resulting dataframe
weekly_forecast_df.head()

,dt,sunrise,sunset,moonrise,moonset,moon_phase,temp,feels_like,pressure,humidity,dew_point,wind_speed,wind_deg,wind_gust,weather,clouds,pop,rain,uvi,snow
0,1677067200,1677049333,1677087438,1677053940,1677098400,0.08,"{'day': 8.87, 'min': 6.21, 'max': 9.17, 'night...","{'day': 6.03, 'night': 3.88, 'eve': 5.9, 'morn...",1014,93,7.63,7.12,330,10.53,"[{'id': 500, 'main': 'Rain', 'description': 'l...",94,0.66,1.25,1.17,NaN
1,1677153600,1677135622,1677173935,1677141360,1677189600,0.12,"{'day': 7.5, 'min': 3.73, 'max': 7.83, 'night'...","{'day': 3.84, 'night': 2.39, 'eve': 2.81, 'mor...",1017,84,4.75,9.05,41,14.73,"[{'id': 500, 'main': 'Rain', 'description': 'l...",100,0.61,1.11,1.24,NaN
2,1677240000,1677221909,1677260431,1677228780,0,0.15,"{'day': 9.09, 'min': 4.43, 'max': 9.55, 'night...","{'day': 6.06, 'night': 3.78, 'eve': 3.61, 'mor...",1012,65,2.63,8.40,44,14.43,"[{'id': 803, 'main': 'Clouds', 'description': ...",78,0.24,NaN,1.75,NaN
3,1677326400,1677308195,1677346927,1677316320,1677280680,0.19,"{'day': 7.58, 'min': 3.39, 'max': 7.58, 'night...","{'day': 3.74, 'night': -0.56, 'eve': 0.84, 'mo...",1016,62,0.45,8.55,49,12.59,"[{'id': 803, 'main': 'Clouds', 'description': ...",69,0.18,NaN,1.25,NaN
4,1677412800,1677394481,1677433422,1677404040,1677371700,0.22,"{'day': 5.86, 'min': 1.2, 'max': 6.17, 'night'...","{'day': 0.85, 'night': -3.96, 'eve': -1.56, 'm...",1024,48,-4.39,9.55,51,14.80,"[{'id': 800, 'main': 'Clear', 'description': '...",8,0.04,NaN,2.04,NaN


In [9]:
# Checking what columns contain data in the string ('object') form
weekly_forecast_df.select_dtypes(include=['object']).head()

,temp,feels_like,weather
0,"{'day': 8.87, 'min': 6.21, 'max': 9.17, 'night...","{'day': 6.03, 'night': 3.88, 'eve': 5.9, 'morn...","[{'id': 500, 'main': 'Rain', 'description': 'l..."
1,"{'day': 7.5, 'min': 3.73, 'max': 7.83, 'night'...","{'day': 3.84, 'night': 2.39, 'eve': 2.81, 'mor...","[{'id': 500, 'main': 'Rain', 'description': 'l..."
2,"{'day': 9.09, 'min': 4.43, 'max': 9.55, 'night...","{'day': 6.06, 'night': 3.78, 'eve': 3.61, 'mor...","[{'id': 803, 'main': 'Clouds', 'description': ..."
3,"{'day': 7.58, 'min': 3.39, 'max': 7.58, 'night...","{'day': 3.74, 'night': -0.56, 'eve': 0.84, 'mo...","[{'id': 803, 'main': 'Clouds', 'description': ..."
4,"{'day': 5.86, 'min': 1.2, 'max': 6.17, 'night'...","{'day': 0.85, 'night': -3.96, 'eve': -1.56, 'm...","[{'id': 800, 'main': 'Clear', 'description': '..."


In [10]:
# Checking the contents of a cell in 'temp' column
print(weekly_forecast_df.loc[0, "temp"])
(print)

# Checking the contents of a cell in 'feels_like' column
print(weekly_forecast_df.loc[0, "feels_like"])

# Checking the contents of a cell in 'weather' column
print(weekly_forecast_df.loc[0, "weather"])

{'day': 8.87, 'min': 6.21, 'max': 9.17, 'night': 6.21, 'eve': 8.39, 'morn': 6.85}
{'day': 6.03, 'night': 3.88, 'eve': 5.9, 'morn': 5.31}
[{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10d'}]


Each of the cells in these columns contains a dictionary. I would like to split this information into several columns, so that each key in such dictionaries becomes a separate column.

In [11]:
# Splitting columns using the previously defined function 'split_dict_to_columns'
split_dict_to_columns(weekly_forecast_df, "temp")

split_dict_to_columns(weekly_forecast_df, "feels_like")

In the 'weather' column, the cell contains a list that contains a dictionary, so there is a slight adjustment to the code compared to other columns.

In [12]:
# Keys for creating columns from splitting 'weather' column
weather_keys = weekly_forecast_df["weather"][0][0].keys()
print(weather_keys)

# Splitting dictionary in column 'weather' into several columns
for row in range(0, len(weekly_forecast_df)): 
    for key in weather_keys:
        weekly_forecast_df.loc[row, "weather_"+key] = weekly_forecast_df["weather"][row][0][key]


dict_keys(['id', 'main', 'description', 'icon'])


In [13]:
# Checking the resulting table (commented to avoid long output)
#pd.set_option('display.max_columns', None)
#weekly_forecast_df.head(5)

In [14]:
print(weekly_forecast_df.columns)

# Dropping some unnecessary columns
columns_to_drop = ['moonrise', 'moonset', 'moon_phase', 'temp',
       'feels_like', 'weather', 'weather_icon']

weather_df = weekly_forecast_df.drop(columns_to_drop, axis=1)

Index(['dt', 'sunrise', 'sunset', 'moonrise', 'moonset', 'moon_phase', 'temp',
       'feels_like', 'pressure', 'humidity', 'dew_point', 'wind_speed',
       'wind_deg', 'wind_gust', 'weather', 'clouds', 'pop', 'rain', 'uvi',
       'snow', 'temp_day', 'temp_min', 'temp_max', 'temp_night', 'temp_eve',
       'temp_morn', 'feels_like_day', 'feels_like_night', 'feels_like_eve',
       'feels_like_morn', 'weather_id', 'weather_main', 'weather_description',
       'weather_icon'],
      dtype='object')


Some columns contain information about time in timestamp format. We will convert this information into datetime format.

In [15]:
timestamp_date_columns = ['dt', 'sunrise', 'sunset']
for column in timestamp_date_columns:
    timestamp_to_datetime(weather_df, column)

'weather_df' dataframe contains 245 rows (35 cities x 7 days). To be able to merge the table easily with the other tables in this project, we will create a column 'city_name' for each of the rows in the 'weather_df'. The weather data was obtained based on the list of cities in the table 'city_coord', so we can simply create a column using the same list with each city name repeated 7 times.

In [16]:
print("Length of the 'weather_df': ", len(weather_df))

# Adding column with city name for each row with daily forecast
target_df_start_index = 0
target_df_end_index = 7
for source_df_row in range(0, len(city_coord)):
    for target_df_row in range(target_df_start_index, target_df_end_index):
        weather_df.loc[target_df_row, "city_name"] = city_coord.loc[source_df_row, "city_name"]
    target_df_start_index = target_df_start_index+7
    target_df_end_index= target_df_end_index+7


# Checking the resulting dataframe (commented to avoid long output)
# weather_df.head(10)

Length of the 'weather_df':  245


### Saving the dataframe locally and in bucket S3

In [17]:
# Saving the dataframe as .csv file locally
weather_df.to_csv("weather_forecast.csv", index=False)

In [19]:
# (Cell is commented to avoid unnecessary rewriting in S3)
"""# Access key for user with access to write in S3 bucket
S3_ACCESS_KEY_ID =  os.getenv("S3_ACCESS_KEY_ID")
# Secret key for user with access to write in S3 bucket 
S3_SECRET_ACCESS_KEY =  os.getenv("S3_SECRET_ACCESS_KEY")

# Writing the .csv file to bucket S3
session = boto3.Session(aws_access_key_id=S3_ACCESS_KEY_ID, 
                      aws_secret_access_key=S3_SECRET_ACCESS_KEY)
s3 = session.resource("s3")
bucket = s3.Bucket("kayak-booking-bucket-12-12-2022") 
bucket.upload_file("weather_forecast.csv", Key="weather_forecast.csv")"""